In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain import hub
from  langchain.schema import Document
import json
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from typing import Iterable

In [4]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [3]:
def load_docs_from_jsonl(file_path)->Iterable[Document]:
    array = []
    with open(file_path, 'r') as jsonl_file:
        for line in jsonl_file:
            data = json.loads(line)
            obj = Document(**data)
            array.append(obj)
    return array
# save_docs_to_jsonl(documents,'chunk_data.jsonl')


documents=load_docs_from_jsonl("fresh_chunk.jsonl")

In [5]:
db = FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=db.as_retriever()

In [14]:
llm=ChatOpenAI(model='gpt-3.5-turbo-0125')

In [15]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000275D4DF3250>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000275D4DF7CA0>, model_name='gpt-3.5-turbo-0125', openai_api_key=SecretStr('**********'), openai_proxy='')

In [16]:
prompt = hub.pull("rlm/rag-prompt")

In [17]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [18]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [19]:
retriever.invoke('where is sastra university is located?')

[Document(page_content='Saraswathi Sadan, the Central Library of SASTRA is an architectural elegance over an area of about 4,340 sq. m with modern study area, audio–visual room, internet and reprographic facilities. It is the nerve centre of the entire campus catering to the literary and knowledge pursuits of the faculty and students. Responding to the technological developments in the 21st century, SASTRA has adopted automation and integrated Information & Communication Technology tools into the resources and services offered in the Central Library. This has ensured that the library does not become an irrelevant', metadata={'source': 'https://www.sastra.edu/infrastructure/library.html '}),
 Document(page_content="SASTRA does not have any other off-campus centre. Please bring to the notice of the admissions office if you find any misuse of SASTRA Deemed University's name in any other location. They are not approved by either Government of India or by SASTRA Deemed University.\r\n\r\n\t

In [33]:
import csv
def extract_column_to_list(csv_file_path, column_index):
    with open(csv_file_path, 'r') as file:
        reader = csv.reader(file)
        return [row[column_index] for row in reader]
# questions = extract_column_to_list('Mistral_test.csv', 0)  # Assuming email is the third column
# references = extract_column_to_list('Mistral_test.csv', 1) 
Mistral_Answers = extract_column_to_list('Mistral_test.csv', 2)
print(Mistral_Answers)

['Mistral_Responce', 'ACRS stands for Accelerated Credit Registration System. It is a system at SASTRA University that allows high-performing students with a minimum CGPA and SGPA of 7.5 to complete their course work requirements by the seventh semester. This enables them to utilize the eighth semester solely for project work, which they can do in academic institutions, R&D centers, industries in India or premier institutions abroad. Additionally, students with a CGPA of 7.5 or higher can register for an open elective course each semester, starting from the fourth semester onwards.', "The context provided does not include information about a person named Muthubalan. Therefore, I'm unable to answer the question about Muthubalan's areas of interest.", "Dr. John Bosco Balaguru R's designation is Associate Professor.", 'SASTRA offers the following B.Tech. programs at their Thanjavur and Kumbakonam campuses:\n\n1. Aerospace Engineering\n2. Bioengineering\n3. Bioinformatics\n4. Biotechnology

In [34]:
Mistral_Answers = Mistral_Answers[1:]

In [22]:
len(questions)

31

In [23]:
questions=questions[1:]

In [24]:
questions

['what is ACRS?',
 'what are areas of interest of muthubalan?',
 'what is the designation of john bosco?',
 'what are the progams offered in B.Tech by sastra?',
 'what is the admission process to get seat in B.Tech in sastra?',
 'what are facilities available in sastra hostel?',
 'what are the prices of single, double, 4 in 1 rooms in hostel?',
 'How to calculate CGPA?',
 'how to calculate SGPA?',
 'what is the contact details of registrar?',
 'how can i contact dean of student affairs sastra?',
 'what are educational qualifications of anila glory?',
 'what is the minimum attendance required for attending semester exam?',
 'If i score more than 50 marks what grade i can get?',
 'what does the letter E means in grading?',
 'what is SAP?',
 'what is L,T,P?',
 'how to contact dean of SEEE?',
 'what is speciality of daksh',
 'what are different events conducted in kuruksastra? who should participate in the event?',
 'tell me about library in sastra',
 'what are different laboratories avail

In [25]:
len(questions)

30

In [27]:
len(references)

31

In [28]:
references=references[1:]

In [29]:
len(references)

30

In [30]:
GPT_Answers = []

In [94]:
for i in range(27,30): #27-29
    for j in range(1):
        query=questions[i]
        gpt_answer = rag_chain.invoke(query)
        GPT_Answers.append(gpt_answer)
        # break

In [95]:
len(GPT_Answers)

30

In [96]:
GPT_Answers

['ACRS stands for Accelerated Course Registration System and is offered to students in the B. Tech. programme who maintain a minimum CGPA and SGPA of 7.5. It allows students to complete their course work requirements by VII semester to focus on project work in the VIII semester. Students with a history of F or E grades are not eligible for ACRS.',
 'Areas of interest of Muthubalan include FACTS and power quality.',
 'Associate Professor',
 'The B.Tech. programs offered by SASTRA include Aerospace Engineering, Bioengineering, Bioinformatics, Biotechnology, Chemical Engineering, Civil Engineering, Computer Science & Business Systems, Computer Science & Engineering, Computer Science & Engineering with specializations in Artificial Intelligence & Data Science and Cyber Security & Block Chain Technology. Admissions to these programs are done by the Office of Admissions of SASTRA based on merit, with no donations or capitation fees required. The Chennai campus also offers 4-Year Honours prog

In [102]:
questions[29]

'what is the email address of betty martin?'

In [103]:
Mistral_Answers[29]

'The email address of Betty Martin is bettymartin@sastra.edu.'

In [104]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('Mistral_test.csv')

In [105]:
# Add a new column to the DataFrame
df = df.assign(GPT_answers = GPT_Answers)

In [106]:
df

,Question,Reference,Mistral_Responce,Hallucination_Score,GPT_answers
0,what is ACRS?,5.5.6 Students in the B. Tech. programme have ...,ACRS stands for Accelerated Credit Registratio...,0.949655,ACRS stands for Accelerated Course Registratio...
1,what are areas of interest of muthubalan?,Click here\n\n\n\n\n\n\n\nDr. UMA .D\nSr. Asst...,The context provided does not include informat...,0.520313,Areas of interest of Muthubalan include FACTS ...
2,what is the designation of john bosco?,Faculty Key List\nSchool of Electrical and Ele...,Dr. John Bosco Balaguru R's designation is Ass...,0.996051,Associate Professor
3,what are the progams offered in B.Tech by sastra?,What full-time programmes does SASTRA offer? \...,SASTRA offers the following B.Tech. programs a...,0.827260,The B.Tech. programs offered by SASTRA include...
4,what is the admission process to get seat in B...,How are admissions done? \r\n\t\r\n\tThe Admis...,The admission to B.Tech. programs at SASTRA is...,0.874227,To get admission to B.Tech. programs in SASTRA...
5,what are facilities available in sastra hostel?,SASTRA has always believed in catering to the ...,The facilities available in SASTRA hostels inc...,0.984837,Sastra hostel provides a completely balanced v...
6,"what are the prices of single, double, 4 in 1 ...",Is hostel facility available?\r\n\t\r\n\tThere...,"Based on the provided context, the hostel has ...",0.553227,"The prices of single, double, and 4-in-1 rooms..."
7,How to calculate CGPA?,9.5 Calculation of Cumulative Grade Point Aver...,To calculate the Cumulative Grade Point Averag...,0.704409,"To calculate CGPA, you need to sum the product..."
8,how to calculate SGPA?,9.4 The average performance of a student in a ...,To calculate the Semester Grade Point Average ...,0.954899,"To calculate SGPA, you need to multiply the cr..."
9,what is the contact details of registrar?,S.No School / Office E.Mail Phone 1 Chancellor...,The contact details for the Registrar's Office...,0.235699,The contact details for the Registrar's Office...


In [107]:
from sentence_transformers import CrossEncoder
model = CrossEncoder('vectara/hallucination_evaluation_model')

c:\Users\dines\Desktop\SASBOT_LANGCHAIN\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [108]:
scores=[]

In [109]:
for i in range(30):
    score=model.predict([references[i],GPT_Answers[i]])
    print(score)
    scores.append(score)

0.99777645
0.6980358
0.99667585
0.8280914
0.91442144
0.9893441
0.02125593
0.68609554
0.91306823
0.6836461
0.22494829
0.9777439
0.9780034
0.41363454
0.4513321
0.42955035
0.052155785
0.45602682
0.99530953
0.99837554
0.998988
0.7121443
0.9963366
0.995522
0.99869186
0.021658286
0.9921218
0.9986753
0.58165675
0.4179921


In [110]:
df = df.assign(GPT_Hallucination_Score = scores)

In [111]:
df.head()

,Question,Reference,Mistral_Responce,Hallucination_Score,GPT_answers,GPT_Hallucination_Score
0,what is ACRS?,5.5.6 Students in the B. Tech. programme have ...,ACRS stands for Accelerated Credit Registratio...,0.949655,ACRS stands for Accelerated Course Registratio...,0.997776
1,what are areas of interest of muthubalan?,Click here\n\n\n\n\n\n\n\nDr. UMA .D\nSr. Asst...,The context provided does not include informat...,0.520313,Areas of interest of Muthubalan include FACTS ...,0.698036
2,what is the designation of john bosco?,Faculty Key List\nSchool of Electrical and Ele...,Dr. John Bosco Balaguru R's designation is Ass...,0.996051,Associate Professor,0.996676
3,what are the progams offered in B.Tech by sastra?,What full-time programmes does SASTRA offer? \...,SASTRA offers the following B.Tech. programs a...,0.827260,The B.Tech. programs offered by SASTRA include...,0.828091
4,what is the admission process to get seat in B...,How are admissions done? \r\n\t\r\n\tThe Admis...,The admission to B.Tech. programs at SASTRA is...,0.874227,To get admission to B.Tech. programs in SASTRA...,0.914421


In [114]:
# Rename the 'old_name' column to 'new_name'
df = df.rename(columns={'GPT_answers': 'GPT_Responce'})

In [115]:
df.head()

,Question,Reference,Mistral_Responce,Mistral_Hallucination_Score,GPT_Responce,GPT_Hallucination_Score
0,what is ACRS?,5.5.6 Students in the B. Tech. programme have ...,ACRS stands for Accelerated Credit Registratio...,0.949655,ACRS stands for Accelerated Course Registratio...,0.997776
1,what are areas of interest of muthubalan?,Click here\n\n\n\n\n\n\n\nDr. UMA .D\nSr. Asst...,The context provided does not include informat...,0.520313,Areas of interest of Muthubalan include FACTS ...,0.698036
2,what is the designation of john bosco?,Faculty Key List\nSchool of Electrical and Ele...,Dr. John Bosco Balaguru R's designation is Ass...,0.996051,Associate Professor,0.996676
3,what are the progams offered in B.Tech by sastra?,What full-time programmes does SASTRA offer? \...,SASTRA offers the following B.Tech. programs a...,0.827260,The B.Tech. programs offered by SASTRA include...,0.828091
4,what is the admission process to get seat in B...,How are admissions done? \r\n\t\r\n\tThe Admis...,The admission to B.Tech. programs at SASTRA is...,0.874227,To get admission to B.Tech. programs in SASTRA...,0.914421


In [117]:
df = df.assign(difference = df['GPT_Hallucination_Score'] - df['Mistral_Hallucination_Score'])

In [119]:
df

,Question,Reference,Mistral_Responce,Mistral_Hallucination_Score,GPT_Responce,GPT_Hallucination_Score,difference
0,what is ACRS?,5.5.6 Students in the B. Tech. programme have ...,ACRS stands for Accelerated Credit Registratio...,0.949655,ACRS stands for Accelerated Course Registratio...,0.997776,0.048122
1,what are areas of interest of muthubalan?,Click here\n\n\n\n\n\n\n\nDr. UMA .D\nSr. Asst...,The context provided does not include informat...,0.520313,Areas of interest of Muthubalan include FACTS ...,0.698036,0.177722
2,what is the designation of john bosco?,Faculty Key List\nSchool of Electrical and Ele...,Dr. John Bosco Balaguru R's designation is Ass...,0.996051,Associate Professor,0.996676,0.000625
3,what are the progams offered in B.Tech by sastra?,What full-time programmes does SASTRA offer? \...,SASTRA offers the following B.Tech. programs a...,0.827260,The B.Tech. programs offered by SASTRA include...,0.828091,0.000831
4,what is the admission process to get seat in B...,How are admissions done? \r\n\t\r\n\tThe Admis...,The admission to B.Tech. programs at SASTRA is...,0.874227,To get admission to B.Tech. programs in SASTRA...,0.914421,0.040194
5,what are facilities available in sastra hostel?,SASTRA has always believed in catering to the ...,The facilities available in SASTRA hostels inc...,0.984837,Sastra hostel provides a completely balanced v...,0.989344,0.004507
6,"what are the prices of single, double, 4 in 1 ...",Is hostel facility available?\r\n\t\r\n\tThere...,"Based on the provided context, the hostel has ...",0.553227,"The prices of single, double, and 4-in-1 rooms...",0.021256,-0.531971
7,How to calculate CGPA?,9.5 Calculation of Cumulative Grade Point Aver...,To calculate the Cumulative Grade Point Averag...,0.704409,"To calculate CGPA, you need to sum the product...",0.686096,-0.018314
8,how to calculate SGPA?,9.4 The average performance of a student in a ...,To calculate the Semester Grade Point Average ...,0.954899,"To calculate SGPA, you need to multiply the cr...",0.913068,-0.041831
9,what is the contact details of registrar?,S.No School / Office E.Mail Phone 1 Chancellor...,The contact details for the Registrar's Office...,0.235699,The contact details for the Registrar's Office...,0.683646,0.447947


In [120]:
AVG_Score= sum(scores)/len(scores)

In [121]:
AVG_Score

0.713975934125483

In [122]:
df.to_csv('Hallucination_Score.csv', index=False)